In [4]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import load_workbook

object_counts = {}
object_ids = {}
time_points = []
counts_over_time = []


meters_per_pixel_width = 0.1  
meters_per_pixel_height = 0.1  
frame_interval = 1  

def calculate_object_size(box):
    width_meters = box[2] * meters_per_pixel_width
    height_meters = box[3] * meters_per_pixel_height
    return width_meters, height_meters

def calculate_object_speed(previous_position, current_position):
    distance_pixels = ((current_position[0] - previous_position[0])**2 + (current_position[1] - previous_position[1])**2)**0.5
    speed_meters_per_second = distance_pixels / frame_interval
    return speed_meters_per_second

size_speed_data = {
    'Object': [],
    'Size (Width x Height, meters)': [],
    'Speed (m/s)': []
}

def process_frame(frame, frame_number):
    global object_counts, object_ids, time_points, counts_over_time
    try:
        bbox, labels, conf = cv.detect_common_objects(frame)
    except ValueError as e:
        print(f"Error in object detection: {e}")
    frame_counts = {}
    
    for label, box in zip(labels, bbox):
        object_id = label.lower()
        
        width, height = calculate_object_size(box)
        
        if object_id in object_ids:
            previous_position = object_ids[object_id]['position']
            speed = calculate_object_speed(previous_position, (box[0], box[1]))
            object_ids[object_id]['position'] = (box[0], box[1])
            object_ids[object_id]['size'] = (width, height)
            
            size_speed_data['Object'].append(object_id.capitalize())
            size_speed_data['Size (Width x Height, meters)'].append(f"{width:.2f} x {height:.2f}")
            size_speed_data['Speed (m/s)'].append(f"{speed:.2f}")
            
            cv2.putText(frame, f"Size: {width:.2f} x {height:.2f} m", (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.putText(frame, f"Speed: {speed:.2f} m/s", (int(box[0]), int(box[1]) - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
        else:
            object_ids[object_id] = {'position': (box[0], box[1]), 'size': (width, height)}

        if object_id in frame_counts:
            frame_counts[object_id] += 1
        else:
            frame_counts[object_id] = 1
    
    output_frame = draw_bbox(frame, bbox, labels, conf)
    counts_string = ', '.join([f'{object_id.capitalize()} ({count})' for object_id, count in frame_counts.items()])
    cv2.putText(output_frame, counts_string, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    time_points.append(frame_number)
    counts_over_time.append(frame_counts)

    return output_frame

video_path = 'people_walk.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('people_walk_detect.avi', fourcc, fps, (frame_width, frame_height))

frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame(frame, frame_number)
    cv2.imshow("Object Detection", processed_frame)
    out.write(processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_number += 1

cap.release()
out.release()
cv2.destroyAllWindows()

import plotly.express as px

df = pd.DataFrame(counts_over_time)
df['Frame Number'] = time_points
fig = px.line(df, x='Frame Number', y=df.columns[:-1], title='Object Count Over Time')
fig.show()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_io.cpp:660: error: (-215:Assertion failed) separator_index < line.size() in function 'cv::dnn::darknet::ReadDarknetFromCfgStream'
